In [2]:
from sort.sort import Sort

mot_tracker = Sort()

In [3]:
from ultralytics import YOLO
import cv2
import numpy as np

In [4]:
from util import get_car, read_license_plate, write_csv

Using CPU. Note: This module is much faster with a GPU.


### Load models

In [5]:
coco_model = YOLO('./models/yolov8n.pt')
license_plate_detector = YOLO('./models/license_plate_detector.pt')

### Load video

In [6]:
capture = cv2.VideoCapture('sample.mp4')

### Read frames

In [ ]:
results = {}

vehicles = [2, 3, 5, 7]

# read frames
frame_nmr = -1
ret = True
while ret:
    frame_nmr += 1
    ret, frame = capture.read()
    if ret and frame_nmr < 3:
        results[frame_nmr] = {}
        # detect vehicles
        detections = coco_model(frame)[0]
        detections_ = []
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = detection
            if int(class_id) in vehicles:
                detections_.append([x1, y1, x2, y2, score])
                
        # track vehicles
        track_ids = mot_tracker.update(np.asarray(detections_))

        # detect license plates
        license_plates = license_plate_detector(frame)[0]
        for license_plate in license_plates.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = license_plate